In [90]:
import numpy as np
import qcodes
import matplotlib.pyplot as plt
from functools import partial    
from  qcodes.plots.qcmatplotlib import MatPlot
from  qcodes.plots.pyqtgraph import QtPlot
from scipy.optimize import curve_fit
import scipy.integrate as integrate
import pandas as pd


class AlazarTech():
    def __init__(self,param):
        self.name = param.get('name')
        self.fs = param.get('sampling_rate')
        self.ts = 1/self.fs
        self.record_length = param.get('record_length')
        self.channel_range = param.get('channel_range')
        
    def alazar_bit2volt(self, signal):
        return ()
    
    
    
adc_param_CryoRX = {
    "name" : 'CryoRX', # For setting ...
    "sampling_rate" : 1e9, # samples/s
    "record_length" : 0.002, # in (s)
    "channel_range" : 0.1 # in (v)
    }

pandas.pd

alazar = AlazarTech(adc_param_CryoRX)
alazar.name
    


NameError: name 'pandas' is not defined